In [48]:
import pandas as pd
import numpy as np
import yfinance as yf
AZPN = yf.Ticker("AZPN")
opt = AZPN.option_chain('2020-07-17')
calls = opt.calls
puts = opt.puts
# print(calls)

In [54]:
#k = calls[calls['inTheMoney'] == False]
l = calls[['strike', 'lastPrice']]
m = l.set_index('strike').to_dict()
n = m['lastPrice'] 
print(n)

{70.0: 51.0, 80.0: 40.6, 95.0: 6.15, 100.0: 2.35, 105.0: 0.8, 110.0: 1.19, 115.0: 1.14, 120.0: 1.5, 125.0: 0.65, 130.0: 1.93, 135.0: 0.5, 140.0: 2.15, 145.0: 0.6, 155.0: 0.05, 160.0: 0.1, 175.0: 0.05, 180.0: 0.05}


In [52]:

def Call_Credit_Spread():
#     Credit spreads should be otm
#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio
#     Change currentRatio with the best available one and save that transcation
    i=0
    best_ratio = {'x':0, 'k':0, 'ratio':((2**31)-1)}
    
    for x in n:
        for k in n:
            if k<=x:
                continue
            else:
                currentRatio = 1
                strikeShort = x
                strikeLong = k
                #Selling the cheapest one and then buying the more expensive one, one by one
                premiumShort = n[x]
                premiumLong = n[k]
                #figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                MaxReward = float('{:,.3f}'.format(premiumShort - premiumLong))
                #print("MaxReward = ", MaxReward)
                MaxRisk = float('{:,.2f}'.format(strikeLong - strikeShort - MaxReward))
                #print("MaxRisk = ", MaxRisk)
                if MaxReward == 0:
                    continue
                else: 
                    Ratio =  float('{:,.2f}'.format(MaxRisk/MaxReward))
                    if Ratio < best_ratio['ratio']:
                        best_ratio['k']=k
                        best_ratio['x']=x
                        best_ratio['ratio']=Ratio
                #print("Ratio = ", Ratio)
        return best_ratio
        



In [53]:
print(Call_Credit_Spread())


{'x': 100.0, 'k': 105.0, 'ratio': 2.23}
